<h2><center>THE CHINESE UNIVERSITY OF HONG KONG<br /> 
    Department of Mathematics<br />
 MATH4280<br />
    Innovation and Design in Big Data Analytics</center></h2>

# Test 1

## Question 1

Enter

In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA

## Do not change these
df = pd.read_csv('face_data.csv')

In this question, we are trying to implement the Nearest Centroid Classifier (or Nearest Mean Classifier) in the principal component space. The purpose is to classify images based on how close they are to the centroids (the mean) of each group of data. To improve the efficiency, we perform this classification on the PCA space.

`df` is a dataframe containing 10 face pictures of each of 40 people. Each picture is $64 \times 64$ pixels and flattened to be stored in the first $4096$ columns of `df`. The last column `target` refers to the assigned number of the person. 

To verify the classification method, we will split the data into two groups (`training_set` and `testing_set`). The overview of this approach is as follows:
 
1. Generate the principal components using `training_set`
2. Calculate the centroids of each group in the PCA space
3. Transform the `testing_set` using the principal components obtained in Step 1
4. Calculate the distances of the PCA-transformed data from the centroids
5. Calculate the accuarcy of this method

## Q1a

We first create a training set and a testing set. Each group has 10 pictures. Among the same group, we would like the first 8 to be included in `training_set` and the remaining 2 in `testing_set`.

Your `training_set` and `testing_set` should be $320\times 4097$ and $80\times 4097$ `pd.DataFrame`'s. Their last columns should be named `target`, representing the group number assigned to each row.

In [ ]:
## answer of 1a

training_set = np.zeros([320,4097])
testing_set = np.zeros([80,4097])

for i,k in enumerate(df.iloc[:,-1].unique()):
    training_set[i*8:(i+1)*8] = df[df['target']==k].iloc[:8].values
    testing_set[i*2:(i+1)*2] = df[df['target']==k].iloc[-2:].values

training_set = pd.DataFrame(training_set)
training_set.rename(columns={4096: 'target'},inplace=True)
testing_set = pd.DataFrame(testing_set)
testing_set.rename(columns={4096: 'target'},inplace=True)



## Q1b

We now create the principal components using the `training_set` appropriately. Select a suitable number of components that captures at least $95\%$ of the total variance. Declare a dataframe `images_training_pca` as your PCA-transformed training data. 

Again, the last column of `images_training_pca` is named `target`, representing the assigned class number.

In [ ]:
## answer of 1b

images_training = training_set.iloc[:,:-1]

pca = PCA(n_components=320)
pca.fit(images_training)
print(f"It can capture 95% variance with {np.where(pca.explained_variance_ratio_.cumsum() > 0.95)[0][0]} principal components.")

pca = PCA(n_components = np.where(pca.explained_variance_ratio_.cumsum() > 0.95)[0][0])
images_training_pca = pca.fit_transform(images_training)
images_training_pca = pd.DataFrame(images_training_pca)
images_training_pca = pd.concat([images_training_pca,training_set['target']],axis=1)

images_training_pca



## Q1c

Create another dataframe `training_pca_centers`. 

Assuming `k` is your number of principal components, your `training_pca_centers` should be of size $40\times (k+1)$.

Each row is the center (the mean) of the PCA-transformed data of the same person and also the number assigned to the person.

In [ ]:
## answer of 1c


training_pca_centers = np.zeros((40, images_training_pca.shape[1]))
for i in range(40):
    training_pca_centers[i,:-1]=images_training_pca[images_training_pca['target']==i].iloc[:,:-1].mean(axis=0)
    training_pca_centers[i,-1]=i
training_pca_centers=pd.DataFrame(training_pca_centers)
column_names=list(training_pca_centers.columns)
column_names[-1]='target'
training_pca_centers.columns=column_names


## Alternatively, 
# training_pca_centers = images_training_pca.groupby('target').mean().reset_index()
# cols = [col for col in training_pca_centers.columns if col != 'target']
# training_pca_centers = training_pca_centers[cols + ['target']]

# training_pca_centers.shape

## Q1d

Create a function `find_class(df,x)` that takes in a dataframe `df` and a 1D array `x`.

The last column of `df` marks a class number. This function compares the $l_2$ distance between each row of `df` and `x`, and then returns the class number of the row in `df` with the shortest distance. 

In [ ]:
## answer of 1d

def find_class(df,x):
    min_tmp = np.inf
    for  _, row in df.iterrows():
        distance = np.linalg.norm(row.values[:-1]-x,2)
        if min_tmp > distance:
            min_tmp = distance
            pred_class = row.values[-1]
    return pred_class


## Q1e

Perform PCA on the testing data. Using `find_class()` to find the predicted class for each row of the transformed testing data. Save all the predictions to a 1D array `pred`. 

In [ ]:
## answer of 1e

images_testing = testing_set.iloc[:,:-1]
images_testing_pca = pca.transform(images_testing)
images_testing_pca = pd.DataFrame(images_testing_pca)
images_testing_pca = pd.concat([images_testing_pca,testing_set['target']],axis=1)
pred = np.zeros(testing_set.shape[0])
for k, row in images_testing_pca.iterrows():
    pred[k] = find_class(training_pca_centers,row.values[:-1])


## Q1f

Calculate the accuracy of the predictions using the following definitions:
$$\text{accuracy} = \frac{\#\text{correct predictions}}{\#\text{testing data}}\times 100\%$$

You should achieve above $75\%$ accuracy.

In [ ]:
## answer of 1f

def give_accuracy(pred,target):
    correct_count = 0
    for k,i in enumerate(pred):
        if i == target[k]:
            correct_count += 1
    return correct_count/pred.shape[0]*100

## Alternatively
# def give_accuracy(pred,target):
#     return np.count_nonzero(pred==target)/pred.shape[0]*100

print(f"Accuracy: {give_accuracy(pred,images_testing_pca['target'].values)}%.")